In [5]:
from pathlib import Path
import pandas as pd

root = Path(r"C:\Users\jorif\OneDrive\سطح المكتب\sen\SeizeIT2")

events_files = sorted(root.rglob("*_task-szMonitoring_*events.tsv"))

print(f"Found {len(events_files)} events files across ALL subjects")

all_seizures = []

for f in events_files:
    try:
        df = pd.read_csv(f, sep="\t")
    except:
        continue

    if "eventType" not in df.columns:
        continue

    sz_df = df[df["eventType"].astype(str).str.startswith("sz")].copy()
    if sz_df.empty:
        continue

    sz_df["recording"] = f.relative_to(root).as_posix()
    sz_df["seizure_end"] = sz_df["onset"] + sz_df["duration"]

    all_seizures.append(
        sz_df[["recording", "onset", "duration", "seizure_end", "eventType"]]
    )

if all_seizures:
    seizures_df = pd.concat(all_seizures, ignore_index=True)

    print(seizures_df.head())

    print("\n================ SEIZURE STATISTICS FOR ALL 125 PATIENTS ================")
    print("Total seizures found:", len(seizures_df))
    print("Mean duration (s):", seizures_df["duration"].mean())
    print("Median duration (s):", seizures_df["duration"].median())
    print("Min duration (s):", seizures_df["duration"].min())
    print("Max duration (s):", seizures_df["duration"].max())

    min_sz = seizures_df["duration"].min()
    print("\n🔥 Recommended minimum window size (based on shortest seizure):", min_sz)

else:
    print("No seizures found across all subjects.")


Found 2850 events files across ALL subjects
                                           recording    onset  duration  \
0  sub-001/ses-01/eeg/sub-001_ses-01_task-szMonit...  57975.0      72.0   
1  sub-001/ses-01/eeg/sub-001_ses-01_task-szMonit...  60968.0      81.0   
2  sub-001/ses-01/eeg/sub-001_ses-01_task-szMonit...  16300.0     143.0   
3  sub-001/ses-01/eeg/sub-001_ses-01_task-szMonit...  28285.0      25.0   
4  sub-002/ses-01/eeg/sub-002_ses-01_task-szMonit...   4046.0      20.0   

   seizure_end               eventType  
0      58047.0            sz_foc_ia_nm  
1      61049.0             sz_foc_a_nm  
2      16443.0       sz_foc_ia_m_tonic  
3      28310.0  sz_foc_a_m_automatisms  
4       4066.0             sz_foc_a_nm  

================ SEIZURE STATISTICS FOR ALL 125 PATIENTS ================
Total seizures found: 883
Mean duration (s): 58.4088335220838
Median duration (s): 33.0
Min duration (s): 2.0
Max duration (s): 962.0

🔥 Recommended minimum window size (based on short

In [4]:
seizures_df["duration_sec"] = seizures_df["duration"].astype(int)

dur_table = (
    seizures_df["duration_sec"]
    .value_counts()          
    .sort_index()           
    .reset_index()
)

dur_table.columns = ["duration_sec", "num_seizures"]

print(dur_table.head(20))   
print("\nNumber of 2-second seizures:", 
      dur_table.loc[dur_table["duration_sec"] == 2, "num_seizures"].sum())


    duration_sec  num_seizures
0              2             2
1              4             7
2              5             8
3              6            11
4              7            15
5              8            18
6              9            26
7             10            36
8             11            23
9             12            42
10            13            35
11            14            34
12            15            21
13            16            15
14            17            17
15            18             6
16            19            10
17            20            11
18            21            17
19            22            13

Number of 2-second seizures: 2


In [3]:
out_path = root / "seizure_duration_counts.csv"
dur_table.to_csv(out_path, index=False)
print("Saved table to:", out_path)


Saved table to: C:\Users\jorif\OneDrive\سطح المكتب\sen\SeizeIT2\seizure_duration_counts.csv
